# Data profiling

In [91]:
from support_functions import log_progress, clean_drugbank, query_chembl
import pandas as pd
import numpy as np

## 1. Lookup known cpd targets

In [53]:
# load perturbation info
pert_info_1 = pd.read_csv('data/GSE92742_Broad_LINCS_pert_info.txt', sep='\t', header=0)
pert_info_2a = pd.read_csv('data/GSE70138_Broad_LINCS_pert_info.txt', sep='\t', header=0)
all_pert_info = pd.concat([pert_info_1, pert_info_2a], ignore_index=True)

In [68]:
# lookup cpd inchi keys
all_cpd_pert_info = all_pert_info[pert_info.pert_type == 'trt_cp']
all_cpd_inchis = all_cpd_pert_info[['pert_id','inchi_key']].set_index('pert_id').copy()
# remove duplicates, store as series
all_cpd_inchis = all_cpd_inchis[~all_cpd_inchis.index.duplicated()].inchi_key

### 1a. Lookup Drugbank targets

In [59]:
# lookup and store drugbank targets as dictionary: { inchi: [targets]}
db_db_path = 'data/full_drugbank_database.xml'
drugbank_target_dict = clean_drugbank(db_db_path)

### 1b. Lookup Chembl targets

In [87]:
# lookup and store chembl targets as dictionary: { inchi: [targets]}
chembl_target_dict = {}

In [ ]:
# query CHEMBL API using cpd inchi_keys to find human targets
for inchi in log_progress(all_cpd_inchis.values, every=1):
    human_target_genes = query_chembl(inchi)
    if len(human_target_genes) > 0:
        chembl_target_dict[inchi] = human_target_genes

In [54]:
# load sig info
all_sig_info = pd.DataFrame.from_csv('checkpoint_files/all_sig_info.csv')

/Users/Nico/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (4,6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [14]:
# remove low-confidence signatures
distil_cutoff = 0.2
self_rank_cutoff = 0.05
sig_info = all_sig_info[(all_sig_info['distil_cc_q75'] >= distil_cutoff) &
                        (all_sig_info['pct_self_rank_q25'] <= self_rank_cutoff)
                       ].reset_index(drop=True)